In [6]:
from lxml import html
from pprint import pprint
import requests
from datetime import datetime, date, timedelta
import json

In [7]:
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)\
                    Chrome/81.0.4044.138 Safari/537.36'}

In [46]:
def get_news_mail():
    
    main_link = 'https://news.mail.ru/'
    response = requests.get(main_link, headers=header)
    dom = html.fromstring(response.text)

    links = dom.xpath("//span/a[contains(@href, 'politics')]/@href")

    news=[]
    
    for i in range(len(links)):
        data = {}
        data['links'] = main_link + links[i]
        link = data['links']
        response = requests.get(link, headers=header)
        dom = html.fromstring(response.text)

        text = dom.xpath("//h1[@class = 'hdr__inner']/text()")
        source = dom.xpath("//span[@class ='note']/a/span/text()")
        dates= dom.xpath("//span[@class='note__text breadcrumbs__text js-ago']/@datetime")
        for j in range(len(source)):
            data['text'] = text[j]
            data['source'] = source[j]
            data['date'] = dates[j].replace(u'T', u' ')
            news.append(data)

    return news

In [47]:
def get_news_lenta():
    main_link = 'https://lenta.ru/'
    response = requests.get(main_link, headers=header)
    dom = html.fromstring(response.text)

    links = dom.xpath("//div/section[contains(@class, 'top-seven')]/div/div[@class = 'item']/a/@href")
    text = dom.xpath("//div/section[contains(@class, 'top-seven')]/div/div[@class = 'item']/a/text()")
    dates = dom.xpath("//div/section[contains(@class, 'top-seven')]/div/div[@class = 'item']/a/time/@datetime")

    news = []
    for i in range(len(links)):
        data = {}
        data['links'] = main_link + links[i]
        data['text'] = text[i].replace(u'\xa0', u' ')
        data['dates'] = dates[i]
        news.append(data)

    return news

In [10]:
def get_news_yandex():
    
    main_link = 'https://yandex.ru'
    response = requests.get(main_link+'/news/', headers=header)
    dom = html.fromstring(response.text)


    link = dom.xpath("//div[@aria-labelledby = 'politics']/a/@href")
    #link = 'https://yandex.ru/news/rubric/politics'

    response = requests.get(link[0], headers=header)
    dom = html.fromstring(response.text)

    links = dom.xpath("//h2[@class = 'story__title']/a/@href")
    text = dom.xpath("//h2[@class = 'story__title']/a/text()")
    source = dom.xpath("//div[@class = 'story__info']/div/text()")

    date_format = '%d-%m-%Y'

    news = []
    for i in range(len(links)):
        data = {}
        data['links'] = main_link + links[i]
        data['text'] = text[i].replace(u'\xa0', u' ')
        if 'вчера' in source[i]:
            data['dates'] = (date.today() - timedelta(days=1)).strftime(date_format)
        else:
            data['dates'] = date.today().strftime(date_format)
        data['source'] = source[i][:-6].replace(u'\xa0', u' ').replace('вчера в', '')
        
        news.append(data)

    return news

In [11]:
news_y = get_news_yandex()
pprint(news_y)

with open("news_yandex.json", "w") as write_file:
    json.dump(news_y, write_file)

[{'dates': '25-05-2020',
  'links': 'https://yandex.ru/news/story/Rogozin_predlozhil_sozdat_novuyu_orbitalnuyu_stanciyu--dfaf1fdbe325051fd1e0983178501317?lr=22&lang=ru&stid=44osqFxm66wEO2g9mWco&persistent_id=98541098&rubric=politics&from=rubric',
  'source': 'RT на русском',
  'text': 'Рогозин предложил создать новую орбитальную станцию'},
 {'dates': '25-05-2020',
  'links': 'https://yandex.ru/news/story/Okolo_16_tysyachi_bojcov_CHVK_Vagnera_pokinuli_zapad_Livii--6e1e7d176cd39d92f930f0dcce941293?lr=22&lang=ru&stid=cRZqqpNqOYOl5Qji&persistent_id=98542462&rubric=politics&from=rubric',
  'source': 'Lenta.ru',
  'text': 'Около 1,6 тысячи бойцов ЧВК Вагнера покинули запад Ливии'},
 {'dates': '25-05-2020',
  'links': 'https://yandex.ru/news/story/Pamfilova_rasskazala_o_vnedrenii_ehlektronnogo_golosovaniya--5791c4edb15c2e2885f2807272002c17?lr=22&lang=ru&stid=kAac9DJ0gVQBVJkXslzl&persistent_id=98491282&rubric=politics&from=rubric',
  'source': 'РИА Новости',
  'text': 'Памфилова рассказала о в

In [52]:
news_m = get_news_mail()
pprint(news_m)

with open("news_mail.json","w") as write_file:
    json.dump(news_m, write_file)

[{'date': '2020-05-24 17:03:54+03:00',
  'links': 'https://news.mail.ru//politics/41909070/',
  'source': 'ТАСС',
  'text': 'Нетаньяху заявил о готовности Израиля противостоять агрессии Ирана'},
 {'date': '2020-05-24 16:38:04+03:00',
  'links': 'https://news.mail.ru//politics/41908824/',
  'source': 'РИА Новости',
  'text': 'Захарова объяснила проблему Договора об открытом небе для США'},
 {'date': '2020-05-24 14:50:33+03:00',
  'links': 'https://news.mail.ru//politics/41908165/',
  'source': 'Коммерсантъ',
  'text': 'Япония планирует полностью отменить режим ЧС с 25 мая'},
 {'date': '2020-05-24 10:22:18+03:00',
  'links': 'https://news.mail.ru//politics/41905853/',
  'source': 'ТАСС',
  'text': 'В Гонконге усилили меры безопасности из-за уличных протестов'}]


In [53]:
news_l = get_news_lenta()
pprint(news_l)

with open("news_lenta.json", "w") as write_file:
    json.dump(news_l, write_file)

[{'dates': ' 22:29, 24 мая 2020',
  'links': 'https://lenta.ru//news/2020/05/24/8/',
  'text': 'Стали известны личности задержанных участников перестрелки в '
          'Москве'},
 {'dates': ' 22:27, 24 мая 2020',
  'links': 'https://lenta.ru//news/2020/05/24/surgeon/',
  'text': 'В Подмосковье умер лечивший пациентов с коронавирусом хирург'},
 {'dates': ' 22:23, 24 мая 2020',
  'links': 'https://lenta.ru//news/2020/05/24/prichina/',
  'text': 'Названа причина перестрелки в Москве'},
 {'dates': ' 22:17, 24 мая 2020',
  'links': 'https://lenta.ru//news/2020/05/24/urgant/',
  'text': 'Ургант признался в любви к белорусскому футбольному клубу и получил '
          'ответ'},
 {'dates': ' 22:11, 24 мая 2020',
  'links': 'https://lenta.ru//news/2020/05/24/popova_vrachi/',
  'text': 'Попова отреагировала на данные о высокой смертности врачей в '
          'России'},
 {'dates': ' 22:00, 24 мая 2020',
  'links': 'https://lenta.ru//news/2020/05/24/propusk/',
  'text': 'В России появился черный р